In [2]:
!pip install "gymnasium==0.28.1"
!pip install stable_baselines3
!pip install swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 74.6 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376096 sha256=750a557288e5b11a11dd95c2f8fd4887dea526b17742c8fd15bca01865fc9435
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
  Attempting uninstall: pygame
    Found existing installation: pygame 2.6.1
    Uninstalling pygame-2.6.1:
      Successfully uninstalled pygame-2.6.1


In [3]:
import os
import json

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3 import TD3
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# TD3 on LunarLanderContinuous-v2 with stable_baselines3

In [5]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model based on the training reward
    and tracking the actor losses, critic losses, and rewards per step.

    :param check_freq: Frequency to check and save model.
    :param log_dir: Directory to save the model.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf
        self.actor_losses = []
        self.critic_losses = []
        self.rewards_per_step = []
        self.episode_reward = 0
        self.num_steps_in_episode = 0

    def _init_callback(self) -> None:
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        # Get the current reward from 'infos'
        reward = self.locals['rewards'][0]  # Reward for the current step
        self.episode_reward += reward
        self.num_steps_in_episode += 1

        # Record reward per step
        self.rewards_per_step.append(reward)

        if self.n_calls % self.check_freq == 0:
            # Retrieve the training reward (mean reward of the last 100 steps)
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                mean_reward = np.mean(y[-100:])
                if self.verbose >= 1:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per step: {mean_reward:.2f}")

                # Save model if a new best reward is found
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    if self.verbose >= 1:
                        print(f"Saving new best model to {self.save_path}")
                    self.model.save(self.save_path)

        # Access and log losses from the logger dictionary
        self.actor_losses.append(self.logger.name_to_value['train/actor_loss'])
        self.critic_losses.append(self.logger.name_to_value['train/critic_loss'])

        return True

In [7]:
results = {}

seeds = [1,2,3,4,5]
callbacks = []
log_dirs = []
models = []

for seed in seeds:
    # Set log directory
    log_dir = f"tmp{seed}/"
    os.makedirs(log_dir, exist_ok=True)

    # Create and wrap the environment
    env = gym.make("LunarLanderContinuous-v2")
    env.reset(seed=seed)
    env = Monitor(env, log_dir)

    # Add action noise for exploration
    n_actions = env.action_space.shape[-1]
    action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

    # Initialize model
    model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=0, device="cuda")

    # Create callback to save model and track losses
    callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

    callbacks.append(callback)
    log_dirs.append(log_dir)
    models.append(model)

    # Train the model
    timesteps = 1e5
    model.learn(total_timesteps=int(timesteps), callback=callback)

    # Stocker les résultats de l'entraînement pour cette seed
    results = {
        "actor_losses": [float(loss) for loss in callback.actor_losses],
        "critic_losses": [float(loss) for loss in callback.critic_losses],
        "rewards_per_step": [float(reward) for reward in callback.rewards_per_step],
    }

    # Sauvegarde des résultats dans un fichier JSON après chaque seed
    with open(f"training_results_seed{seed}.json", "w") as f:
        json.dump(results, f, indent=4)

    print(f"Les résultats de l'entraînement pour seed={seed} ont été enregistrés dans training_results_seed{seed}.json.")

Num timesteps: 1000
Best mean reward: -inf - Last mean reward per step: -545.48
Saving new best model to tmp1/best_model
Num timesteps: 2000
Best mean reward: -545.48 - Last mean reward per step: -429.08
Saving new best model to tmp1/best_model
Num timesteps: 3000
Best mean reward: -429.08 - Last mean reward per step: -396.68
Saving new best model to tmp1/best_model
Num timesteps: 4000
Best mean reward: -396.68 - Last mean reward per step: -377.72
Saving new best model to tmp1/best_model
Num timesteps: 5000
Best mean reward: -377.72 - Last mean reward per step: -371.27
Saving new best model to tmp1/best_model
Num timesteps: 6000
Best mean reward: -371.27 - Last mean reward per step: -358.07
Saving new best model to tmp1/best_model
Num timesteps: 7000
Best mean reward: -358.07 - Last mean reward per step: -347.85
Saving new best model to tmp1/best_model
Num timesteps: 8000
Best mean reward: -347.85 - Last mean reward per step: -336.57
Saving new best model to tmp1/best_model
Num timeste

In [10]:
results_part2 = {}

seeds = [6,7,8,9,10]
callbacks = []
log_dirs = []
models = []

for seed in seeds:
    # Set log directory
    log_dir = f"tmp{seed}/"
    os.makedirs(log_dir, exist_ok=True)

    # Create and wrap the environment
    env = gym.make("LunarLanderContinuous-v2")
    env.reset(seed=seed)
    env = Monitor(env, log_dir)

    # Add action noise for exploration
    n_actions = env.action_space.shape[-1]
    action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

    # Initialize model
    model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=0, device="cuda")

    # Create callback to save model and track losses
    callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

    callbacks.append(callback)
    log_dirs.append(log_dir)
    models.append(model)

    # Train the model
    timesteps = 1e5
    model.learn(total_timesteps=int(timesteps), callback=callback)

    # Stocker les résultats de l'entraînement pour cette seed
    results = {
        "actor_losses": [float(loss) for loss in callback.actor_losses],
        "critic_losses": [float(loss) for loss in callback.critic_losses],
        "rewards_per_step": [float(reward) for reward in callback.rewards_per_step],
    }

    # Sauvegarde des résultats dans un fichier JSON après chaque seed
    with open(f"training_results_seed{seed}.json", "w") as f:
        json.dump(results, f, indent=4)

    print(f"Les résultats de l'entraînement pour seed={seed} ont été enregistrés dans training_results_seed{seed}.json.")

Num timesteps: 1000
Best mean reward: -inf - Last mean reward per step: -783.99
Saving new best model to tmp10/best_model
Num timesteps: 2000
Best mean reward: -783.99 - Last mean reward per step: -688.45
Saving new best model to tmp10/best_model
Num timesteps: 3000
Best mean reward: -688.45 - Last mean reward per step: -599.44
Saving new best model to tmp10/best_model
Num timesteps: 4000
Best mean reward: -599.44 - Last mean reward per step: -585.61
Saving new best model to tmp10/best_model
Num timesteps: 5000
Best mean reward: -585.61 - Last mean reward per step: -541.33
Saving new best model to tmp10/best_model
Num timesteps: 6000
Best mean reward: -541.33 - Last mean reward per step: -490.16
Saving new best model to tmp10/best_model
Num timesteps: 7000
Best mean reward: -490.16 - Last mean reward per step: -458.31
Saving new best model to tmp10/best_model
Num timesteps: 8000
Best mean reward: -458.31 - Last mean reward per step: -441.28
Saving new best model to tmp10/best_model
Num